<a href="https://colab.research.google.com/github/meetAmarAtGithub/15_Reva_Speech_Analytics/blob/main/Day4_1_Identify_Word_Game_Using_Google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install SpeechRecognition

# all imports
#For Audio File
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript
#For speech recognition
import random
import time
import speech_recognition as sr

#Javascript to record audio from microphone
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""
def recognize_speech_from_audio_file(recognizer, audio_file):
    """
    Transcribe speech from an audio file.

    Returns a dictionary with three keys:
    - "success": a boolean indicating whether or not the API request was successful
    - "error": `None` if no error occurred, otherwise a string containing an error message if the API could not be reached or speech was unrecognizable
    - "transcription": `None` if speech could not be transcribed, otherwise a string containing the transcribed text
    """
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be a `Recognizer` instance")

    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    try:
        with sr.AudioFile(audio_file) as source:
            audio = recognizer.record(source)
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        response["error"] = "Unable to recognize speech"

    return response

#Record audio from Microphone
'''
One can't use the mic from Google Colab directly as we do with our own machine. One have to use JavaScript to let the browser enable the mic.
'''
def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream

if __name__ == "__main__":
    WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
    NUM_GUESSES = 3
    PROMPT_LIMIT = 5

    recognizer = sr.Recognizer()

    word = random.choice(WORDS)

    instructions = (
        "I'm thinking of one of these words:\n"
        "{words}\n"
        "You have {n} tries to guess which one.\n"
    ).format(words=', '.join(WORDS), n=NUM_GUESSES)

    print(instructions)
    time.sleep(3)

    for i in range(NUM_GUESSES):
        for j in range(PROMPT_LIMIT):
            print('Guess {}. Provide the path to an audio file:'.format(i + 1))
            audio_file = record(5)
            guess = recognize_speech_from_audio_file(recognizer, audio_file)
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
            print("Unable to transcribe speech. Please try again.")

        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break

        print("You said: {}".format(guess["transcription"]))

        guess_is_correct = guess["transcription"].lower() == word.lower()
        user_has_more_attempts = i < NUM_GUESSES - 1

        if guess_is_correct:
            print("Correct! You win!".format(word))
            break
        elif user_has_more_attempts:
            print("Incorrect. Try again.\n")
        else:
            print("Sorry, you lose!\nI was thinking of '{}'.".format(word))
            break


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
I'm thinking of one of these words:
apple, banana, grape, orange, mango, lemon
You have 3 tries to guess which one.

Guess 1. Provide the path to an audio file:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording !


AssertionError: ignored

In [41]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd "/content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics"

!pip install SpeechRecognition

# all imports
import random
import time
import speech_recognition as sr
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript
import wave

# Set the path to your Google Drive folder
AUDIO_SAVE_PATH = "/content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics"

# Javascript to record audio from microphone
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def recognize_speech_from_audio_file(recognizer, audio_file):
    """
    Transcribe speech from an audio file.

    Returns a dictionary with three keys:
    - "success": a boolean indicating whether or not the API request was successful
    - "error": `None` if no error occurred, otherwise a string containing an error message if the API could not be reached or speech was unrecognizable
    - "transcription": `None` if speech could not be transcribed, otherwise a string containing the transcribed text
    """
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be a `Recognizer` instance")

    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    try:
        with sr.AudioFile(audio_file) as source:
            audio = recognizer.record(source)
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        response["error"] = "Unable to recognize speech"

    return response

# Record audio from Microphone
def record(sec=3):
    print("Speak Now...")
    display(Javascript(RECORD))
    sec += 1
    s = output.eval_js('record(%d)' % (sec*1000))
    print("Done Recording !")
    b = b64decode(s.split(',')[1])

    # Save the audio file to Google Drive
    audio_file_path = AUDIO_SAVE_PATH + "audio.wav"
    with wave.open(audio_file_path, "wb") as f:
        f.setnchannels(1)  # Set the number of channels (mono)
        f.setsampwidth(2)  # Set the sample width (16-bit)
        f.setframerate(16000)  # Set the sample rate (16 kHz)
        f.writeframes(b)

    return audio_file_path

if __name__ == "__main__":
    WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
    NUM_GUESSES = 3
    PROMPT_LIMIT = 5

    recognizer = sr.Recognizer()

    word = random.choice(WORDS)

    instructions = (
        "I'm thinking of one of these words:\n"
        "{words}\n"
        "You have {n} tries to guess which one.\n"
    ).format(words=', '.join(WORDS), n=NUM_GUESSES)

    print(instructions)
    time.sleep(3)

    for i in range(NUM_GUESSES):
        for j in range(PROMPT_LIMIT):
            print('Guess {}. Speak now:'.format(i + 1))
            audio_file = record(5)
            guess = recognize_speech_from_audio_file(recognizer, audio_file)
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
            print("Unable to transcribe speech. Please try again.")

        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break

        print("You said: {}".format(guess["transcription"]))

        guess_is_correct = guess["transcription"].lower() == word.lower()
        user_has_more_attempts = i < NUM_GUESSES - 1

        if guess_is_correct:
            print("Correct! You win!")
            break
        elif user_has_more_attempts:
            print("Incorrect. Try again.\n")
        else:
            print("Sorry, you lose!\nI was thinking of '{}'.".format(word))
            break


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
I'm thinking of one of these words:
apple, banana, grape, orange, mango, lemon
You have 3 tries to guess which one.

Guess 1. Speak now:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
Unable to transcribe speech. Please try again.
Guess 1. Speak now:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
Unable to transcribe speech. Please try again.
Guess 1. Speak now:
Speak Now...


...

Done Recording !
Unable to transcribe speech. Please try again.
Guess 1. Speak now:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
Unable to transcribe speech. Please try again.
Guess 1. Speak now:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
Unable to transcribe speech. Please try again.
ERROR: Unable to recognize speech


In [34]:
!pip install SpeechRecognition

import random
import time
import speech_recognition as sr
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record_audio(duration):
    """
    Record audio for the specified duration (in seconds) and return the audio data in byte format.
    """
    print("Speak Now...")
    display(Javascript(RECORD))
    sec = int(duration) + 1
    s = output.eval_js('record(%d)' % (sec * 1000))
    print("Done Recording!")

    try:
        audio_bytes = b64decode(s.split(',')[1])
        return audio_bytes
    except Exception as e:
        print("An error occurred while decoding the audio: {}".format(e))
        return None


def recognize_speech_from_audio_byte(recognizer, audio_byte):
    """
    Transcribe speech from audio data in byte format.

    Returns a dictionary with three keys:
    - "success": a boolean indicating whether or not the API request was successful
    - "error": `None` if no error occurred, otherwise a string containing an error message if the API could not be reached or speech was unrecognizable
    - "transcription": `None` if speech could not be transcribed, otherwise a string containing the transcribed text
    """
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be a `Recognizer` instance")

    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    try:
        audio = sr.AudioData(audio_byte, sample_rate=16000, sample_width=2)  # Modify the sample_rate and sample_width if necessary
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError as e:
        response["success"] = False
        response["error"] = "API request error: {}".format(e)
    except sr.UnknownValueError:
        response["error"] = "Speech could not be transcribed"
    except Exception as e:
        response["error"] = "An error occurred: {}".format(e)

    return response



if __name__ == "__main__":
    WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
    NUM_GUESSES = 3
    PROMPT_LIMIT = 5

    recognizer = sr.Recognizer()

    word = random.choice(WORDS)

    instructions = (
        "I'm thinking of one of these words:\n"
        "{words}\n"
        "You have {n} tries to guess which one.\n"
    ).format(words=', '.join(WORDS), n=NUM_GUESSES)

    print(instructions)
    time.sleep(3)

    for i in range(NUM_GUESSES):
        for j in range(PROMPT_LIMIT):
            print('Guess {}. Speak now:'.format(i + 1))
            audio_byte = record_audio(2)  # Record audio in byte format
            guess = recognize_speech_from_audio_byte(recognizer, audio_byte)
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
            print("Unable to transcribe speech. Please try again.")

        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break

        print("You said: {}".format(guess["transcription"]))

        guess_is_correct = guess["transcription"].lower() == word.lower()
        user_has_more_attempts = i < NUM_GUESSES - 1

        if guess_is_correct:
            print("Correct! You win!".format(word))
            break
        elif user_has_more_attempts:
            print("Incorrect. Try again.\n")
        else:
            print("Sorry, you lose!\nI was thinking of '{}'.".format(word))
            break

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
I'm thinking of one of these words:
apple, banana, grape, orange, mango, lemon
You have 3 tries to guess which one.

Guess 1. Speak now:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording!
Unable to transcribe speech. Please try again.
Guess 1. Speak now:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording!
Unable to transcribe speech. Please try again.
Guess 1. Speak now:
Speak Now...


...

Done Recording!
Unable to transcribe speech. Please try again.
Guess 1. Speak now:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording!
Unable to transcribe speech. Please try again.
Guess 1. Speak now:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording!
Unable to transcribe speech. Please try again.
ERROR: Speech could not be transcribed


In [37]:
!pip install vosk

import random
import time
import vosk
import sys
import os
import wave

def recognize_speech_from_audio_byte(recognizer, audio_byte):
    """
    Transcribe speech from audio data in byte format using the Vosk library.

    Returns a dictionary with three keys:
    - "success": a boolean indicating whether or not the speech recognition was successful
    - "error": `None` if no error occurred, otherwise a string containing an error message if the recognition failed
    - "transcription": `None` if speech could not be transcribed, otherwise a string containing the transcribed text
    """
    if not isinstance(recognizer, vosk.KaldiRecognizer):
        raise TypeError("`recognizer` must be a `vosk.KaldiRecognizer` instance")

    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    try:
        recognizer.AcceptWaveform(audio_byte)
        response["transcription"] = recognizer.Result()
    except Exception as e:
        response["success"] = False
        response["error"] = "Speech recognition failed: {}".format(e)

    return response

def record_audio(duration):
    """
    Record audio for the specified duration (in seconds) and return the audio data in byte format.
    """
    print("Speak Now...")
    display(Javascript(RECORD))
    sec = int(duration) + 1
    s = output.eval_js('record(%d)' % (sec * 1000))
    print("Done Recording!")

    try:
        audio_bytes = b64decode(s.split(',')[1])
        return audio_bytes
    except Exception as e:
        print("An error occurred while decoding the audio: {}".format(e))
        return None

if __name__ == "__main__":
    # Set the list of words, maximum number of guesses, and prompt limit
    WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
    NUM_GUESSES = 3
    PROMPT_LIMIT = 5

    # Download and extract the Vosk model
    !wget https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
    !unzip vosk-model-small-en-us-0.15.zip

    # Create a Vosk recognizer instance
    model_path = "vosk-model-small-en-us-0.15"
    if not os.path.exists(model_path):
        sys.stderr.write(f'Model folder not found in {model_path}.')
        sys.exit(1)
    model = vosk.Model(model_path)
    sample_rate = 16000
    recognizer = vosk.KaldiRecognizer(model, sample_rate)

    # Get a random word from the list
    word = random.choice(WORDS)

    # Format the instructions string
    instructions = (
        "I'm thinking of one of these words:\n"
        "{words}\n"
        "You have {n} tries to guess which one.\n"
    ).format(words=', '.join(WORDS), n=NUM_GUESSES)

    # Show instructions and wait 3 seconds before starting the game
    print(instructions)
    time.sleep(3)

    for i in range(NUM_GUESSES):
        for j in range(PROMPT_LIMIT):
            print('Guess {}. Speak now:'.format(i + 1))
            audio_byte = record_audio(5)  # Record audio in byte format
            guess = recognize_speech_from_audio_byte(recognizer, audio_byte)
            if guess["transcription"]:
                break

        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break

        print("You said: {}".format(guess["transcription"]))

        guess_is_correct = guess["transcription"].strip().lower() == word.lower()
        user_has_more_attempts = i < NUM_GUESSES - 1

        if guess_is_correct:
            print("Correct! You win!")
            break
        elif user_has_more_attempts:
            print("Incorrect. Try again.")
        else:
            print("Sorry, you're out of guesses. The word was '{}'".format(word))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-06-17 03:36:40--  https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41205931 (39M) [application/zip]
Saving to: ‘vosk-model-small-en-us-0.15.zip.1’

vosk-model-small-en 100%[===================>]  39.30M  19.6MB/s    in 2.0s    

2023-06-17 03:36:43 (19.6 MB/s) - ‘vosk-model-small-en-us-0.15.zip.1’ saved [41205931/41205931]

Archive:  vosk-model-small-en-us-0.15.zip
replace vosk-model-small-en-us-0.15/am/final.mdl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: vosk-model-small-en-us-0.15/am/final.mdl  
replace vosk-model-small-en-us-0.15/graph/disambig_tid.int? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: vosk-model-small-en-us-0.15/gra

<IPython.core.display.Javascript object>

Done Recording!
You said: {
  "text" : ""
}
Incorrect. Try again.
Guess 2. Speak now:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording!
You said: {
  "text" : ""
}
Incorrect. Try again.
Guess 3. Speak now:
Speak Now...


<IPython.core.display.Javascript object>

Done Recording!
You said: {
  "text" : ""
}
Sorry, you're out of guesses. The word was 'orange'
